
<center> <h1> <u> <font color='red'> AI Powered Language Processing </font> </u> </h1> </center>

#Session: Text classification using ML


###Objective:
 1. *Apply Tf-idf.*
 2. *Apply LR Algorithm.*





In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sklearn
print(sklearn.__version__)

1.5.2


In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
np.random.seed(42)

CPU times: user 3.87 s, sys: 382 ms, total: 4.25 s
Wall time: 11.9 s


#Dataset Fetching

In [ ]:
# Download the data
!wget -O reviews.csv https://www.dropbox.com/s/vyhodtbbumedlv2/restaurant-reviews.csv?dl=0

--2024-11-18 16:06:49--  https://www.dropbox.com/s/vyhodtbbumedlv2/restaurant-reviews.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/2ca73mc0dykpss9phrj8t/restaurant-reviews.csv?rlkey=t4stjed1v7e9e6h5hucqmvdha&dl=0 [following]
--2024-11-18 16:06:49--  https://www.dropbox.com/scl/fi/2ca73mc0dykpss9phrj8t/restaurant-reviews.csv?rlkey=t4stjed1v7e9e6h5hucqmvdha&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce1621bff855391a481d8594161.dl.dropboxusercontent.com/cd/0/inline/CenwwanzbQzP2UzvT40Y0gaOTCqtnULoV6tdxeQmxYIDvYozNIAQXiNevcORw7ivK43RZSpQcLFDqOW4TzmboLriz1qtmFkFmaMRsTSrPn4ajUGt7U8fZ1qyDVu9iv2XBdLSlEDPzVSjhh49g9wRe_Kz/file# [following]
--2024-11-18 16:06:50--  https://uce1621bff855391a481d85

In [ ]:
# %%time
data = pd.read_csv("reviews.csv")
data = data.dropna()

In [ ]:
tprint("Size of the total data==> ", len(data))
class_names= data.Sentiment.unique() ## Get the unique values in a specific column
print(class_names)

Size of the total data==>  1431
['positive' 'negative']


In [ ]:
data['training_label']=data.Sentiment.replace({'positive':1,'negative':0})
data.sample(20)

,Reviews,Sentiment,training_label
123,গ্রেট জায়গা ... তাদের আচরণ খুব ভাল ... চলতে থ...,positive,1
1191,স্টাফ ব্যবহার এত খারাপ ছিল। আচরণ অত্যন্ত untol...,negative,0
462,খাদ্য চমৎকার আচরণ,positive,1
351,ফুড কোয়ালিটি আগের থেকে অনেক বাজে সার্ভিস ঠিক আ...,positive,1
1296,"আমি জায়গা উপায় এটা পছন্দ করি। , উষ্ণ বন্ধুত্...",negative,0
665,খাবারটা ঠিক ছিল কিন্তু তারা আমাকে এককথায় লুট ক...,negative,0
757,কিন্তু থাই অবশ্যই স্পষ্টভাবে মহান নয় - নমনীয...,negative,0
527,আমি এই জায়গা এন খাদ্য ভাল আচরণ সঙ্গে ভালো খাব...,positive,1
70,যুক্তিসংগত দামের সাথে চমৎকার মানের খাবার ও সেব...,positive,1
554,গুড খাদ্য কিন্তু পাওয়া সব আইটেম না,positive,1


In [ ]:
%%time
# text cleaning

def cleaning_bangla(x):
  x = re.sub('[^\u0980-\u09FF]',' ', x)
  # Remove quotes from source text
  x = re.sub("'", '', x)
  # create a set of all special characters
  x = x.strip()
  return x

data['cleaned'] = data['Reviews'].apply(cleaning_bangla)

CPU times: user 27.7 ms, sys: 2.84 ms, total: 30.6 ms
Wall time: 35.3 ms


In [ ]:
data.head(15)

,Reviews,Sentiment,training_label,cleaned
0,পিজাটা চমৎকার ছিল । আমি এবং আমার বন্ধুদের এটি ...,positive,1,পিজাটা চমৎকার ছিল আমি এবং আমার বন্ধুদের এটি ...
1,এরাবিয়ান মাস্টারের অভ্যন্তরীণ সজ্জাটা অনন্য। ...,positive,1,এরাবিয়ান মাস্টারের অভ্যন্তরীণ সজ্জাটা অনন্য ...
2,কোরিয়ান এবং জামাইকান BBQ উপভোগ করার জন্য একটি...,positive,1,কোরিয়ান এবং জামাইকান উপভোগ করার জন্য একটি...
3,BBQ এর মেনুগুলো সেরা।,positive,1,এর মেনুগুলো সেরা
4,শ্রেষ্ঠ স্বাদ ..... খাবারের মানও ভাল ....,positive,1,শ্রেষ্ঠ স্বাদ খাবারের মানও ভাল
5,সেরা পরিবেশ সঙ্গে শ্রেষ্ঠ স্বাদ। একেবারে প্রিম...,positive,1,সেরা পরিবেশ সঙ্গে শ্রেষ্ঠ স্বাদ একেবারে প্রিম...
6,সত্যিই খেতে দারুণ টেস্ট !!! আমি শুধু BBQ ভালব...,positive,1,সত্যিই খেতে দারুণ টেস্ট আমি শুধু ভালব...
7,মহান খাদ্য বাহ ভাল বাহ সঙ্গে মহান সেবা ..........,positive,1,মহান খাদ্য বাহ ভাল বাহ সঙ্গে মহান সেবা
8,অসাধারণ এবং সুস্বাদু খাদ্য ... ভাল সার্ভিস এবং...,positive,1,অসাধারণ এবং সুস্বাদু খাদ্য ভাল সার্ভিস এবং...
9,গরম এবং মসলাযুক্ত চিকেন ফ্রাইটা পছন্দনীয়। চমৎক...,positive,1,গরম এবং মসলাযুক্ত চিকেন ফ্রাইটা পছন্দনীয় চমৎক...


##Visualize tf-idf and count vectorizer

In [ ]:
tf_idf = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(),  use_idf = True)
tf_corpus = tf_idf.fit_transform(data.cleaned[:3])
cv = CountVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split())
cv_corpus = cv.fit_transform(data.cleaned[:3])

In [ ]:
#Here first parameter is the number of documents and second parameter is the number of features

print(tf_corpus.shape, cv_corpus.shape)

print(cv.vocabulary_)

print(cv.get_feature_names_out())

(3, 31) (3, 31)
{'পিজাটা': 23, 'চমৎকার': 16, 'ছিল': 17, 'আমি': 3, 'এবং': 9, 'আমার': 2, 'বন্ধুদের': 25, 'এটি': 7, 'পছন্দ': 22, 'হয়েছে': 30, 'এরাবিয়ান': 10, 'মাস্টারের': 27, 'অভ্যন্তরীণ': 1, 'সজ্জাটা': 28, 'অনন্য': 0, 'এটির': 8, 'খাদ্য': 14, 'সুস্বাদু': 29, 'একটু': 6, 'মসলাযুক্ত': 26, 'তাদের': 21, 'কর্মীরা': 12, 'খুব': 15, 'বন্ধুত্বপূর্ণ': 24, 'কোরিয়ান': 13, 'জামাইকান': 19, 'উপভোগ': 4, 'করার': 11, 'জন্য': 18, 'একটি': 5, 'জায়গা': 20}
['অনন্য' 'অভ্যন্তরীণ' 'আমার' 'আমি' 'উপভোগ' 'একটি' 'একটু' 'এটি' 'এটির'
 'এবং' 'এরাবিয়ান' 'করার' 'কর্মীরা' 'কোরিয়ান' 'খাদ্য' 'খুব' 'চমৎকার' 'ছিল'
 'জন্য' 'জামাইকান' 'জায়গা' 'তাদের' 'পছন্দ' 'পিজাটা' 'বন্ধুত্বপূর্ণ'
 'বন্ধুদের' 'মসলাযুক্ত' 'মাস্টারের' 'সজ্জাটা' 'সুস্বাদু' 'হয়েছে']


In [ ]:
# Convert sparse matrix to dataframe
cv_corpus = pd.DataFrame.sparse.from_spmatrix(cv_corpus)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in cv.vocabulary_.items()}
# Rename each column using the mapping
for col in cv_corpus.columns:
    cv_corpus.rename(columns={col: col_map[col]}, inplace=True)
cv_corpus

,অনন্য,অভ্যন্তরীণ,আমার,আমি,উপভোগ,একটি,একটু,এটি,এটির,এবং,...,তাদের,পছন্দ,পিজাটা,বন্ধুত্বপূর্ণ,বন্ধুদের,মসলাযুক্ত,মাস্টারের,সজ্জাটা,সুস্বাদু,হয়েছে
0,0,0,1,1,0,0,0,1,0,1,...,0,1,1,0,1,0,0,0,0,1
1,1,1,0,0,0,0,1,0,1,1,...,1,0,0,1,0,1,1,1,1,0
2,0,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(tf_idf.vocabulary_) ## Returns the vocabulary with index

print(tf_idf.get_feature_names_out()) ## Return the feature words


{'পিজাটা': 23, 'চমৎকার': 16, 'ছিল': 17, 'আমি': 3, 'এবং': 9, 'আমার': 2, 'বন্ধুদের': 25, 'এটি': 7, 'পছন্দ': 22, 'হয়েছে': 30, 'এরাবিয়ান': 10, 'মাস্টারের': 27, 'অভ্যন্তরীণ': 1, 'সজ্জাটা': 28, 'অনন্য': 0, 'এটির': 8, 'খাদ্য': 14, 'সুস্বাদু': 29, 'একটু': 6, 'মসলাযুক্ত': 26, 'তাদের': 21, 'কর্মীরা': 12, 'খুব': 15, 'বন্ধুত্বপূর্ণ': 24, 'কোরিয়ান': 13, 'জামাইকান': 19, 'উপভোগ': 4, 'করার': 11, 'জন্য': 18, 'একটি': 5, 'জায়গা': 20}
['অনন্য' 'অভ্যন্তরীণ' 'আমার' 'আমি' 'উপভোগ' 'একটি' 'একটু' 'এটি' 'এটির'
 'এবং' 'এরাবিয়ান' 'করার' 'কর্মীরা' 'কোরিয়ান' 'খাদ্য' 'খুব' 'চমৎকার' 'ছিল'
 'জন্য' 'জামাইকান' 'জায়গা' 'তাদের' 'পছন্দ' 'পিজাটা' 'বন্ধুত্বপূর্ণ'
 'বন্ধুদের' 'মসলাযুক্ত' 'মাস্টারের' 'সজ্জাটা' 'সুস্বাদু' 'হয়েছে']


In [ ]:
# Convert sparse matrix to dataframe
tf_corpus = pd.DataFrame.sparse.from_spmatrix(tf_corpus)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in tf_idf.vocabulary_.items()}
# Rename each column using the mapping
for col in tf_corpus.columns:
    tf_corpus.rename(columns={col: col_map[col]}, inplace=True)
tf_corpus

,অনন্য,অভ্যন্তরীণ,আমার,আমি,উপভোগ,একটি,একটু,এটি,এটির,এবং,...,তাদের,পছন্দ,পিজাটা,বন্ধুত্বপূর্ণ,বন্ধুদের,মসলাযুক্ত,মাস্টারের,সজ্জাটা,সুস্বাদু,হয়েছে
0,0,0,0.334689,0.334689,0,0,0,0.334689,0,0.197673,...,0,0.334689,0.334689,0,0.334689,0,0,0,0,0.334689
1,0.263993,0.263993,0,0,0,0,0.263993,0,0.263993,0.155918,...,0.263993,0,0,0.263993,0,0.263993,0.263993,0.263993,0.263993,0
2,0,0,0,0,0.355173,0.355173,0,0,0,0.209771,...,0,0,0,0,0,0,0,0,0,0


#Model Training

In [ ]:
corpus, labels = data.cleaned, data.training_label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus,labels,train_size = 0.8, test_size = 0.2,random_state =0)

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
%%time
vec = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(), use_idf=True)
# vec = CountVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split())
clf = MultinomialNB()
pipe = make_pipeline(vec, clf)
pipe.fit(X_train, y_train)

CPU times: user 37.9 ms, sys: 73 µs, total: 38 ms
Wall time: 44.5 ms


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7c4e7443f5b0>)),
                ('multinomialnb', MultinomialNB())])

In [ ]:
from sklearn import metrics
def print_report(pipe):
    y_pred = pipe.predict(X_test)
    report = metrics.classification_report(y_test, y_pred,
        target_names=['negative','positive'])
    cm= confusion_matrix(y_test, y_pred)
    print(report)
    print(cm)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe)

              precision    recall  f1-score   support

    negative       0.81      0.91      0.86       155
    positive       0.88      0.75      0.81       132

    accuracy                           0.84       287
   macro avg       0.84      0.83      0.83       287
weighted avg       0.84      0.84      0.83       287

[[141  14]
 [ 33  99]]
accuracy: 0.836


In [ ]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(['negative','positive'], y_pred):
        print("{:.3f} {}".format(prob, target))

doc = data.cleaned[1]
print(doc)
print_prediction(doc)

এরাবিয়ান মাস্টারের  অভ্যন্তরীণ সজ্জাটা অনন্য  এটির খাদ্য সুস্বাদু এবং একটু মসলাযুক্ত    তাদের কর্মীরা খুব বন্ধুত্বপূর্ণ
0.254 negative
0.746 positive


#Model Explanation

In [ ]:
%%time
vec = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(), use_idf=True)
# vec = CountVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split())
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(X_train, y_train)

CPU times: user 593 ms, sys: 10 ms, total: 604 ms
Wall time: 1.17 s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7c4e743af0a0>)),
                ('logisticregressioncv', LogisticRegressionCV())])